# Web Scraping Homework - Mission to Mars

In [ ]:
#import dependencies

from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)